In [ ]:
import sys
sys.path.append("/home/artem.lobantsev/pyproj/brain-texts/src/")
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data import mimic_cxr_dataset as cxr

from torch.utils.data import Dataset

In [ ]:
DATASET_PATH = "/ws/rif-net/data/origin/mimic-cxr-2.0.0/"
CHEXPERT_CSV_PATH = "/ws/rif-net/data/origin/mimic-cxr-2.0.0/mimic-cxr-2.0.0-chexpert.csv"

In [ ]:
dataset = cxr.MIMIC_CXR_Dataset(DATASET_PATH, CHEXPERT_CSV_PATH)

In [ ]:
errors = []
for subj in tqdm(dataset):
    for study in subj:
        for image in study:
            try:
                _ = image.image
                study.report
                study.chexpert_labels
            except Exception as e:
                errors.append((subj.id, study.id, image.id, e))

In [ ]:
df = pd.read_csv("/ws/rif-net/data/origin/mimic-cxr-2.0.0/mimic-cxr-2.0.0-chexpert.csv")
df10 = df[df['subject_id'].apply(lambda x: str(x).startswith('10'))]
df10[df10.Pneumonia.notna()]["study_id"].values

In [ ]:
labels = df10[df10['Pneumonia'].notna()][["study_id", 'Pneumonia']].astype('int')

In [ ]:
labels[labels["Pneumonia"] != -1]

In [ ]:
class MIMICCXRTorchDataset(Dataset):
    def __init__(self, root_path: str, cxr_chexpert_csv_path: str, label_name: str = None):

        df = pd.read_csv(cxr_chexpert_csv_path)
        df10 = df[df['subject_id'].apply(lambda x: str(x).startswith('10'))]
        chexpert_labels = df10[df10[label_name].notna()]
        chexpert_labels = chexpert_labels[["study_id", label_name]].astype('int')
        chexpert_labels = chexpert_labels[chexpert_labels[label_name] != -1]

        self.study_ids = list(chexpert_labels.values[:, 0])
        self.labels = list(chexpert_labels.values[:, 1])
        self.mimic_dataset = cxr.MIMIC_CXR_Dataset(root_path, cxr_chexpert_csv_path)
    
    def __len__(self):
        return len(self.study_ids)
    
    def __getitem__(self):
        